In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
%matplotlib inline

In [2]:
# First I want to read in the full dataset and remove duplicate entries.
# The dataset author has said that articles are duplicated for each tag (this makes no sense LOL).

In [3]:
medium = pd.read_csv('Medium_AggregatedData.csv')
medium.head()

,audioVersionDurationSec,codeBlock,codeBlockCount,collectionId,createdDate,createdDatetime,firstPublishedDate,firstPublishedDatetime,imageCount,isSubscriptionLocked,...,slug,name,postCount,author,bio,userId,userName,usersFollowedByCount,usersFollowedCount,scrappedDate
0,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,blockchain,Blockchain,265164.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
1,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,samsung,Samsung,5708.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
2,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,it,It,3720.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
3,0,NaN,0.0,NaN,2018-01-07,2018-01-07 17:04:37,2018-01-07,2018-01-07 17:06:29,13,False,...,technology,Technology,166125.0,George Sykes,NaN,93b9e94f08ca,tasty231,6.0,22.0,20181104
4,0,NaN,0.0,NaN,2018-01-07,2018-01-07 17:04:37,2018-01-07,2018-01-07 17:06:29,13,False,...,robotics,Robotics,9103.0,George Sykes,NaN,93b9e94f08ca,tasty231,6.0,22.0,20181104


In [12]:
medium.columns

Index(['audioVersionDurationSec', 'codeBlock', 'codeBlockCount',
       'collectionId', 'createdDate', 'createdDatetime', 'firstPublishedDate',
       'firstPublishedDatetime', 'imageCount', 'isSubscriptionLocked',
       'language', 'latestPublishedDate', 'latestPublishedDatetime',
       'linksCount', 'postId', 'readingTime', 'recommends',
       'responsesCreatedCount', 'socialRecommendsCount', 'subTitle',
       'tagsCount', 'text', 'title', 'totalClapCount', 'uniqueSlug',
       'updatedDate', 'updatedDatetime', 'url', 'vote', 'wordCount',
       'publicationdescription', 'publicationdomain',
       'publicationfacebookPageName', 'publicationfollowerCount',
       'publicationname', 'publicationpublicEmail', 'publicationslug',
       'publicationtags', 'publicationtwitterUsername', 'tag_name', 'slug',
       'name', 'postCount', 'author', 'bio', 'userId', 'userName',
       'usersFollowedByCount', 'usersFollowedCount', 'scrappedDate'],
      dtype='object')

In [10]:
# Obviously there are a ton of duplicate entries here

medium[['title', 'uniqueSlug', 'tag_name', 'slug', 'name', 'language']].head(10)

,title,uniqueSlug,tag_name,slug,name,language
0,"Private Business, Government and Blockchain",private-business-government-and-blockchain-100...,Blockchain,blockchain,Blockchain,en
1,"Private Business, Government and Blockchain",private-business-government-and-blockchain-100...,Samsung,samsung,Samsung,en
2,"Private Business, Government and Blockchain",private-business-government-and-blockchain-100...,It,it,It,en
3,EPQ draft 1 (4844 words),introduction-3-1000c43bcb97,Technology,technology,Technology,en
4,EPQ draft 1 (4844 words),introduction-3-1000c43bcb97,Robotics,robotics,Robotics,en
5,EPQ draft 1 (4844 words),introduction-3-1000c43bcb97,Automation,automation,Automation,en
6,EPQ draft 1 (4844 words),introduction-3-1000c43bcb97,Epq,epq,Epq,en
7,"Ascent of data Science, SAS and Big data Analy...",ascent-of-data-science-sas-and-big-data-analys...,Data Science,data-science,Data Science,en
8,"Ascent of data Science, SAS and Big data Analy...",ascent-of-data-science-sas-and-big-data-analys...,Sas Training In Pune,sas-training-in-pune,Sas Training In Pune,en
9,"Ascent of data Science, SAS and Big data Analy...",ascent-of-data-science-sas-and-big-data-analys...,Hadoop Training Mumbai,hadoop-training-mumbai,Hadoop Training Mumbai,en


In [13]:
# Create a list of unique titles
# Then for each title, create a list of tags it contains
# Add these tuples to a list
# Delete duplicate titles from the dataset, then add the list of tag-tuples as a new column
# This will reduce the size of the dataset by ~75%

titles = medium['title'].unique()
len(titles)

70276

In [14]:
# 'Only' 70,276 unique articles in this dataset

In [15]:
tags = medium['tag_name'].unique()
len(tags)

19278

In [24]:
# The dataset contains 19,278 tags!

In [35]:
# Function to find all tags for a certain title and return a list of tags
def find_tags(title):
    rows = medium[medium['title'] == title]
    tags = list(rows['tag_name'].values)
    return tags

In [37]:
find_tags('Private Business, Government and Blockchain')

['Blockchain', 'Samsung', 'It']

In [38]:
%timeit find_tags('Private Business, Government and Blockchain')

11.1 ms ± 368 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [39]:
# This will take a very long time on the full dataset.
# Lets drop rows of non-english articles first

medium = medium[medium['language'] == 'en']

In [43]:
tag_dict = {'titles': [], 'tags': []}

for title in titles:
    tag_dict['titles'].append(title)
    tag_dict['tags'].append(find_tags(title))

In [44]:
tag_df = pd.DataFrame(tag_dict)

In [50]:
medium = medium.drop_duplicates(subset = 'title', keep = 'first')

In [68]:
def addTags(title):
    try:
        tags = list(tag_df[tag_df['titles'] == title]['tags'])[0]
    except:
        tags = np.NaN
    return tags

In [ ]:
medium['allTags'] = [addTags(x) for x in medium['title']]

In [72]:
medium.reset_index(drop = True, inplace = True)

In [73]:
medium[['title', 'allTags']].head()

,title,allTags
0,"Private Business, Government and Blockchain","[Blockchain, Samsung, It]"
1,EPQ draft 1 (4844 words),"[Technology, Robotics, Automation, Epq]"
2,"Ascent of data Science, SAS and Big data Analy...","[Data Science, Sas Training In Pune, Hadoop Tr..."
3,Can a robot love us better than another human ...,"[Robotics, Meditation, Therapy, Artificial Int..."
4,"2017 Big Data, AI and IOT Use Cases","[Artificial Intelligence, Data Science, Big Da..."


In [76]:
# the number of rows has now been greatly reduced.
medium.shape

(64417, 51)

In [81]:
# It seems that there are still many redundant columns.
# Drop all the date columns and only keep the datetime columns

medium.drop(columns = ['createdDate', 'firstPublishedDate', 'latestPublishedDate', 'updatedDate'], inplace = True)

In [82]:
medium.columns

Index(['audioVersionDurationSec', 'codeBlock', 'codeBlockCount',
       'collectionId', 'createdDatetime', 'firstPublishedDatetime',
       'imageCount', 'isSubscriptionLocked', 'language',
       'latestPublishedDatetime', 'linksCount', 'postId', 'readingTime',
       'recommends', 'responsesCreatedCount', 'socialRecommendsCount',
       'subTitle', 'tagsCount', 'text', 'title', 'totalClapCount',
       'uniqueSlug', 'updatedDatetime', 'url', 'vote', 'wordCount',
       'publicationdescription', 'publicationdomain',
       'publicationfacebookPageName', 'publicationfollowerCount',
       'publicationname', 'publicationpublicEmail', 'publicationslug',
       'publicationtags', 'publicationtwitterUsername', 'tag_name', 'slug',
       'name', 'postCount', 'author', 'bio', 'userId', 'userName',
       'usersFollowedByCount', 'usersFollowedCount', 'scrappedDate',
       'allTags'],
      dtype='object')

In [118]:
# drop some more unnecessary columns

# audioVersionDurationSec is unneccessary
# language is all english
# tagsCount is inaccurate
# uniqueSlug is just a combination of title and postId
# vote is just whether the dataset aggregator clapped
# I really doubt I'll need the publication facebook page name, domain, email, slug, or twitter
# tag_name is now unneccessary
# slug and name are redundant with tags
# postCount is just the number of posts with a tag, meaningless now
# userId is mostly meaningless. Already have author name and userName
# scrappedDate is just 20181104 for every article

medium.drop(columns = ['audioVersionDurationSec', 'language', 'tagsCount', 'uniqueSlug', 'vote',
                      'publicationfacebookPageName', 'publicationpublicEmail', 'publicationdomain',
                      'publicationslug', 'publicationtwitterUsername', 'tag_name', 'slug', 'name',
                      'postCount', 'userId', 'scrappedDate'], inplace = True)

In [119]:
# Number of columns has now been reduced by 40%
medium.shape

(64417, 31)

In [120]:
medium.columns

Index(['codeBlock', 'codeBlockCount', 'collectionId', 'createdDatetime',
       'firstPublishedDatetime', 'imageCount', 'isSubscriptionLocked',
       'latestPublishedDatetime', 'linksCount', 'postId', 'readingTime',
       'recommends', 'responsesCreatedCount', 'socialRecommendsCount',
       'subTitle', 'text', 'title', 'totalClapCount', 'updatedDatetime', 'url',
       'wordCount', 'publicationdescription', 'publicationfollowerCount',
       'publicationname', 'publicationtags', 'author', 'bio', 'userName',
       'usersFollowedByCount', 'usersFollowedCount', 'allTags'],
      dtype='object')

In [122]:
medium.to_csv('medium_dataset.csv', index = False)

In [164]:
medium = pd.read_csv('medium_dataset.csv')

list